In [1]:
# 3 modalities

In [ ]:
!git clone https://github.com/Zafar-Lab/multiHIVE.git

In [4]:
import scanpy as sc
import scvi
import sys
sys.path.append("/home/anirudhn/Krushna/ATAC_Support/multiHIVE/")
from src.model import multiHIVE
import torch
rc_parms = {"figure.figsize": [4, 4], "figure.dpi": 300, "font.size": 10, "font.family": "Arial"}
save_parms = {"bbox_inches": "tight", "transparent": True}

In [5]:
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)
torch.set_float32_matmul_precision("high")

Seed set to 0


Last run with scvi-tools version: 1.3.0


In [6]:
adata = sc.read_h5ad( "/home/anirudhn/Krushna/Data/TEA-seq/TEA-seq.h5ad")
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 25517 × 165454
    obs: 'cell_type', 'batch'
    var: 'modality'
    obsm: 'protein_expression'

In [7]:
# adata = scvi.data.organize_multiome_anndatas(adata)
adata = adata[:, adata.var["modality"].argsort()].copy()
sc.pp.filter_genes(adata, min_cells=int(adata.shape[0] * 0.01))
multiHIVE.setup_anndata(adata, batch_key="batch", protein_expression_obsm_key = "protein_expression")
adata

INFO     Using column names from columns of adata.obsm['protein_expression']                                       


/tmp/ipykernel_554711/4167032945.py:4: DeprecationWarning: multiHIVE is supposed to work with MuData. the use of anndata is deprecated and will be removed in scvi-tools 1.4. Please use setup_mudata
  multiHIVE.setup_anndata(adata, batch_key="batch", protein_expression_obsm_key = "protein_expression")


AnnData object with n_obs × n_vars = 25517 × 48872
    obs: 'cell_type', 'batch', '_indices', '_scvi_batch', '_scvi_labels'
    var: 'modality', 'n_cells'
    uns: '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'protein_expression'

In [8]:
adata.obsm['protein_expression']

,CD10,CD11b,CD11c,CD123,CD127,CD14,CD141,CD16,CD172a,CD185,...,FceRI,HLA.DR,IgD,IgG1.K.Isotype.Control,IgM,KLRG1,TCR.Va24.Ja18,TCR.Va7.2,TCR.a.b,TCR.g.d
CAATGTCAGTGAACGA-6,2,10,250,32,2,51,1,11,143,1,...,0,56,4,0,2,6,0,1,16,43
GGATATTGTTTGTGGA-6,1,31,511,19,3,149,13,4,254,0,...,2,122,3,1,21,7,1,0,21,78
AGGCCCAGTGTCCAAA-6,2,11,403,9,0,70,34,5,279,1,...,13,189,9,1,4,18,1,1,15,118
CTCCTGAGTTGCACAA-6,1,10,372,11,1,84,16,4,165,2,...,2,62,4,0,1,12,1,0,13,34
ATTAGCGGTTAGGCTA-6,5,8,207,43,4,57,13,14,133,7,...,8,211,2,10,2,65,5,1,21,203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GGAGCGATCGGTCATG-3,1,0,6,23,0,10,3,3,1,3,...,3,131,35,0,1,8,2,0,6,22
CCGTTACTCATTTAGG-3,1,1,22,1,4,7,1,75,4,1,...,0,5,4,0,1,40,0,0,14,21
CAGGATGCACTCGCTC-3,1,3,636,76,2,10,32,193,283,2,...,0,61,2,0,3,14,6,0,13,40
GGTCAAGCAGCTAATT-3,1,4,84,0,3,3,5,271,10,0,...,2,13,3,0,1,8,1,0,16,21


In [9]:
vae = multiHIVE(adata, latent_distribution="normal",
                n_genes=(adata.var["modality"] == "Gene Expression").sum(),
                n_regions=(adata.var["modality"] == "Peaks").sum(),
                n_proteins=46
               )
vae.train()

# generated_data = vae.posterior_predictive_sample(adata, swap_latent=False)
# rna_sample = generated_data[:,:hvg].copy()
# proteins_sample = pd.DataFrame(generated_data[:,hvg:],  index= adata.obs_names, columns = adata.obsm['protein_counts'].columns)
# adata.obsm['RNA_Z1_denoised'] = rna_sample
# adata.obsm['protein_Z1_denoised'] = proteins_sample

# generated_data = vae.posterior_predictive_sample(adata, swap_latent=True)
# rna_sample = generated_data[:,:hvg].copy()
# proteins_sample = pd.DataFrame(generated_data[:,hvg:],  index= adata.obs_names, columns = adata.obsm['protein_counts'].columns)
# adata.obsm['RNA_Z2_denoised'] = rna_sample
# adata.obsm['protein_Z2_denoised'] = proteins_sample

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Encoder(
  (encoder_r_1): Sequential(
    (0): Linear(in_features=48922, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
  )
  (z_mean_encoder_delta_1): Linear(in_features=128, out_features=20, bias=True)
  (z_var_encoder_delta_1): Linear(in_features=128, out_features=20, bias=True)
  (encoder_r_2): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_sta

/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:317: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.
/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may b

Training:   0%|          | 0/314 [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
vae.get_latent_representation()

In [ ]:
vae.save("./outputs/tea-seq/saved_model/", save_anndata=True, overwrite = True)

In [ ]:
with plt.rc_context(rc_parms):
    print("Zp_multiHIVE")
    sc.pp.neighbors(adata, use_rep='Z_multiHIVE')
    sc.tl.umap(adata)
    adata.obsm["Z1p_umap"] = adata.obsm['X_umap']
    ax = sc.pl.umap(adata,color=['cell_type'], frameon= False, return_fig = True, title='')
    plt.savefig("./outputs/tea-seq/Z1p_umap-cell.png", **save_parms)
    ax = sc.pl.umap(adata,color=['batch'], frameon= False, return_fig = True, title='')  
    plt.savefig("./outputs/tea-seq/Z1p_umap-batch.png", **save_parms)

    print("Z1r_totalVI")
    sc.pp.neighbors(adata, use_rep='Zr_multiHIVE')
    sc.tl.umap(adata)
    adata.obsm["Z1r_umap"] = adata.obsm['X_umap']
    ax = sc.pl.umap(adata,color=['cell_type'], frameon= False, return_fig = True, title='')
    plt.savefig("./outputs/tea-seq/Z1r_umap-cell.png", **save_parms)
    ax = sc.pl.umap(adata,color=['batch'], frameon= False, return_fig = True, title='')  
    plt.savefig("./outputs/tea-seq/Z1r_umap-batch.png", **save_parms)

    print("Z2_totalVI")
    sc.pp.neighbors(adata, use_rep='Z2_multiHIVE')
    sc.tl.umap(adata)
    adata.obsm["Z2_umap"] = adata.obsm['X_umap']
    ax = sc.pl.umap(adata,color=['cell_type'], frameon= False, return_fig = True, title='')
    plt.savefig("./outputs/tea-seq/Z2_umap-cell.png", **save_parms)
    ax = sc.pl.umap(adata,color=['batch'], frameon= False, return_fig = True, title='')  
    plt.savefig("./outputs/tea-seq/Z2_umap-batch.png", **save_parms)
    
    print("Zc_totalVI")
    sc.pp.neighbors(adata, use_rep='Z_multiHIVE')
    sc.tl.umap(adata)
    adata.obsm["Zc_umap"] = adata.obsm['X_umap']
    ax = sc.pl.umap(adata,color=['cell_type'], frameon= False, return_fig = True, title='')
    plt.savefig("./outputs/tea-seq/Zc_umap-cell.png", **save_parms)
    ax = sc.pl.umap(adata,color=['batch'], frameon= False, return_fig = True, title='')  
    plt.savefig("./outputs/tea-seq/Zc_umap-batch.png", **save_parms)

In [ ]:
adata.write("./outputs/tea-seq/multiHIVE.h5ad")

In [17]:
import scib
import numpy as np
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score
import pandas as pd

adata = sc.read_h5ad("./outputs/tea-seq/multiHIVE.h5ad")
adata.obsm['latent'] = adata.obsm["Z_multiHIVE"]
adata = adata[~adata.obs['cell_type'].isna(),:]
scores = []
nmi_df = []
ari_df = []

sc.pp.neighbors(adata,use_rep="latent")
for res in np.linspace(0.1, 2, 20):
    sc.tl.leiden(adata, resolution = res, key_added="Z_leiden")
    n_clusters = np.unique(adata.obs['Z_leiden']).shape[0]
    y_pred = adata.obs['Z_leiden']
    nmi_val, ari_val =  normalized_mutual_info_score(y_pred,adata.obs['cell_type'].values),adjusted_rand_score(y_pred,adata.obs['cell_type'].values)
    nmi_df.append(["NMI", nmi_val])
    ari_df.append(["ARI", ari_val])
    print("Resolution {}, NMI: {}, ARI: {}".format(res,nmi_val, ari_val))

asw_val = scib.me.silhouette(adata, label_key="cell_type", embed="latent")
print("ASW", asw_val)

sc.pp.neighbors(adata, use_rep="latent")
gc_val = scib.me.graph_connectivity(adata, label_key="cell_type")
print("Graph_Connectivity", gc_val)

asw_batch_val = scib.me.silhouette_batch(adata, batch_key="batch", label_key="cell_type", embed="latent")
print("ASW_batch", asw_batch_val)

scores = nmi_df
scores.extend(ari_df)
scores.append(["ASW", asw_val])
scores.append(["Graph Connectivity", gc_val])
scores.append(["ASW-batch", asw_batch_val])

scores = pd.DataFrame(scores)
scores

/tmp/ipykernel_142450/4056882742.py:15: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata, resolution = res, key_added="Z_leiden")


Resolution 0.1, NMI: 0.6763406896172494, ARI: 0.3981271559313612
Resolution 0.2, NMI: 0.7133633371280814, ARI: 0.5265761321922324
Resolution 0.3, NMI: 0.7181898279372012, ARI: 0.5348844985902367
Resolution 0.4, NMI: 0.719798386709889, ARI: 0.5475243296777929
Resolution 0.5, NMI: 0.7431169386639745, ARI: 0.6144222729562159
Resolution 0.6, NMI: 0.7224544145901742, ARI: 0.6303424642262488
Resolution 0.7, NMI: 0.7218772311693429, ARI: 0.6311020041063592
Resolution 0.7999999999999999, NMI: 0.7219543027268239, ARI: 0.6321569681304412
Resolution 0.8999999999999999, NMI: 0.7145215841971732, ARI: 0.6223951619244557
Resolution 0.9999999999999999, NMI: 0.713447681567484, ARI: 0.6245846961091645
Resolution 1.0999999999999999, NMI: 0.6799996835381283, ARI: 0.573959274994151
Resolution 1.2, NMI: 0.6460542993414713, ARI: 0.4872673926736489
Resolution 1.3, NMI: 0.6406617907753422, ARI: 0.48110715561967576
Resolution 1.4, NMI: 0.6269599852042076, ARI: 0.43639010595518213
Resolution 1.5, NMI: 0.62412863

/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/home/anirudhn/anaconda3/envs/multiVI/lib/python3.11/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future 

Graph_Connectivity 0.9030829620422546
mean silhouette per group:                   silhouette_score
group                             
B.Activated               0.987053
B.Naive                   0.991929
DC.Myeloid                0.915947
Mono.CD14                 0.990823
Mono.CD16                 0.961547
NK                        0.989239
Platelets                 0.880082
T.CD4.Memory              0.995157
T.CD4.Naive               0.993368
T.CD8.Effector            0.986678
T.CD8.Naive               0.987734
T.DoubleNegative          0.889414
ASW_batch 0.9640810141837101


,0,1
0,NMI,0.676341
1,NMI,0.713363
2,NMI,0.718190
3,NMI,0.719798
4,NMI,0.743117
5,NMI,0.722454
6,NMI,0.721877
7,NMI,0.721954
8,NMI,0.714522
9,NMI,0.713448


In [18]:
scores.to_csv("./outputs/tea-seq/metrics.csv")

In [19]:
scores_ra = pd.read_csv("../ATAC/outputs/tea-seq/metrics.csv", index_col = 0)